In [1]:
#to do
#+pull data from csv (postcodes - solar 2013) - done
#+changes postcodes into long/lat coords - done
#+plot long/lat coords on map - done
#-create marker details, way to display information
#-figure out how to port to aws server

## Neccessary installations, work in this notebook just need to be uncommented
#!pip install folium
#!pip install pgeocode 
#changing default location of jupyter (to make csv's work easily): https://shanyitan.medium.com/how-to-change-the-working-directory-of-jupyter-and-jupyter-lab-on-windows-environment-bbe5a5a99f05

import pandas as pd
import folium
import pgeocode as pg
import csv

#pgeocode global control, setting AU database
nomi = pg.Nominatim('AU')

%cd /Users/----/Repo/solarForecasting 
#change this depending on working directory, couldn't find a way to easily change to repo directory universally

[WinError 3] The system cannot find the path specified: '/Users/----/Repo/solarForecasting'
C:\Users\Catherine\Desktop\Monash\0. 2021\1. Sem 1\TRC4200\Group Project\Repo\solarForecasting


In [2]:
##creating the initial map
m = folium.Map(location=[-33.3438627,151.4919677], tiles="Stamen Terrain", zoom_start = 8) #zoom 6 displays all of nsw centred on sydney, might want to be closer for detail view

##coverting postcodes to lat/long
#reading postcodes 
pc = [] #creating empty list to kickstart loop

#data for map use
locData = []
lat = []
long = []
locName = []
locInfo = []


with open('data/2012-2013 Solar home electricity data v2.csv', "r") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for lines in csv_reader:
        if '2012-2013 Solar' in lines[0]:
            continue
        if lines[0] == 'Customer':
            header = lines
        else:
            # only using generation data
            if lines[2] not in pc: #detect when changes in postcode, only save each unique
                pc.append(lines[2])

#looping through the list of postcodes to create lat/long lists
for i in range(1,len(pc)): 
    locData = nomi.query_postal_code(pc[i])
    lat.append(locData.latitude)
    long.append(locData.longitude)
    locName.append(locData.place_name) #might be easier to save the correct name from scraping and read from there with postcode data
    locInfo.append('Meaningful data goes in here')
    i = i+1
    
#print(long,lat,locName)
#print(pc)

In [3]:
##creating map data and displaying
# Make a data frame with dots to show on the map - will be done by pulling from csv
data = pd.DataFrame({
   'lon':long,
   'lat':lat,
   'name':locName,
   'info':locInfo, #still needs to be filled with whatever we want to display
}, dtype=str)

# add marker one by one on the map
for i in range(0,len(data)):
   folium.Marker(
      location=[data.iloc[i]['lat'], data.iloc[i]['lon']],
      tooltip=data.iloc[i]['name'],
      icon=folium.Icon(color="blue"),
      popup = data.iloc[i]['info'],
   ).add_to(m)
# Show the map again
m

In [4]:
p1 = [151.23190342392576,-33.989852287056074,]
p2 = [151.1522525450195,-34.08373862378379,]
p3 = [151.01011692490232,-34.01319132053207,]
p4 = [151.01217686142576,-33.95283824086755,]

p5 = [151.278595318457,-33.971062553956976]
p6 = [150.8226627012695,-33.885602355176935]
p7 = [150.883087505957,-33.675577204498786]
p8 = [151.37060581650388,-33.71328264651719]

poly = [p1,p2,p3,p4,p1]
print(poly)

#poly.addTo(m)

#Creating GeoJson file with relevant coordinates
test = { "type": "FeatureCollection",
  "features": [
    { "type": "Feature",
       "geometry": {
         "type": "Polygon",
         "coordinates": [
           poly
           ]
       },
       "properties": {
         "prop0": "Sydney",
         "prop1": {"45.6kwH": "50"}
         }
       }
    ]
  }
#end GeoJson - coordinates need to go above here

#qs = folium.GeoJson(
#    test, #GeoJson file to use
#    name="geojson",
#    tooltip=folium.GeoJsonTooltip( #hover tooltip
#        fields=['prop0','prop1'], #variables, properties from GeoJson file
#        aliases=['Region: ','Generator Capacity'], #titles for properties
#        localize=True)
#).add_to(m)

#js.add_child(folium.Popup("Waterfall")) #setting the popup data for the GeoJson

#folium.LayerControl().add_to(m)
#js.add_to(m) #adding GeoJson with popup data to the map

m


[[151.23190342392576, -33.989852287056074], [151.1522525450195, -34.08373862378379], [151.01011692490232, -34.01319132053207], [151.01217686142576, -33.95283824086755], [151.23190342392576, -33.989852287056074]]


In [8]:
##Looping through different regions code
numRegions = 2 
reg1 = [p1,p2,p3,p4,p1] #hardcoding two regions to test
reg2 = [p5,p6,p7,p8,p5]

#region = list()
#region.append(reg1)
#region.append(reg2)

region = [reg2,reg1]

#print(region)

regionNames = ["Reid","Sydney"]

for i in range(numRegions):
    #Creating GeoJson file with relevant coordinates
    regionJson = { "type": "FeatureCollection",
      "features": [
        { "type": "Feature",
           "geometry": {
             "type": "Polygon",
             "coordinates": [
               region[i]
               ]
           },
           "properties": {
             "name": regionNames[i],
             "data": {"45.6kwH": "50"}
             }
           }
        ]
      }
    #end GeoJson - coordinates need to go above here

    js = folium.GeoJson(
        regionJson, #GeoJson file to use
        name="geojson",
        tooltip=folium.GeoJsonTooltip( #hover tooltip
            fields=['name','data'], #variables, properties from GeoJson file
            aliases=['Region: ','Generator Capacity'], #titles for properties
            localize=True)
    ).add_to(m)

    js.add_child(folium.Popup("Graph Data Json")) #setting the popup data for the GeoJson
    print(i)
    print('added')

    folium.LayerControl().add_to(m)
    js.add_to(m) #adding GeoJson with popup data to the map
    
    print(js)
    
    
#folium.LayerControl().add_to(m)
#js.add_to(m) #adding GeoJson with popup data to the map

#js.add_to(m)
m


0
added
1
added
